### Closures for functional values
* 내가 한국에 있다면 여권이 필요없지만 외국에 가면 국적 확인을 위해 여권이 필요
  * Closure as Passport
  * 코드와 환경을 합친 개념
* 함수값의 의미를 보존하기 위해서 코드 자체만 이동하면 안된다.
  * 함수가 만들어진 환경의 의미를 포함: ***Closure***

In [34]:
                                // incorrect (w.o. closures)
                                // []
val t = 0                       // [t=0]
val f: Int=>Int = {             // [t=0]:[]
    val t = 10                  // [t=0]:[t=10]
    def g(x: Int) : Int = x + t // [t=0]:[t=10, g=(x)x+t]
    g _                         // [t=0]:[t=10, g=(x)x+t]
}                               // [t=0, f=(x)x+t]
f(20)                           // [t=0, f=(x)x+t], 20
                          // f(20) [t=0, f=(x)x+t]:[x=20]

t: Int = 0
f: Int => Int = ammonite.$sess.cmd33$Helper$$Lambda$2416/807056619@4fc741fa
res33_2: Int = 30

* underscore _ : DEF to VALUE by packaging ENV

In [1]:
                                // E1[]
val t = 0                       // E1[t=0]
val f: Int=>Int = {             // E1[t=0]:E2[]
    val t = 10                  // E1[t=0]:E2[t=10]
    def g(x: Int) : Int = x + t // E1[t=0]:E2[t=10, g=(x)x+t]
    g _                         // E1[t=0]:E2[t=10, g=(x)x+t]
}                               // E1[t=0, f=((x)x+t, E2)]
f(20)                           // E1[t=0, f=((x)x+t, E2)], 30
                          // f(20) E1[t=0, f=((x)x+t, E2)]:E2[t=10, g=(x)x+t]:E3[x=20]

t: Int = 0
f: Int => Int = ammonite.$sess.cmd0$Helper$$Lambda$1734/1226621901@2f8b3a8b
res0_2: Int = 30

In [4]:
{
    def f(x:Int) = g(x)
    def g(x: Int) = 10
    val x = f(10)
    x
}

defined function f
defined function g
x: Int = 10
res3_3: Int = 10

In [5]:
{
    // forward reference error from g(x)
    def f(x:Int) = g(x)
    val x = f(10)
    def g(x: Int) = 10
    x
}

defined function f
x: Int = 10
defined function g
res4_3: Int = 10

### Safety Checking
* For `val x = e`, all names in `e` should be defined before this definition.
* For `def x = e`, all names in `e` should be defined before the next `val` definition.
  * *Why? It is the earliest moment when f can be being used.*

In [6]:
def sum1(x: Int): Int =
    if (x <= 0) 0 else x + sum2(x-1)
def sum2(x: Int): Int =
    if (x <= 0) 0 else x + sum1(x-1)
val x = sum1(10)
val y = sum2(20)
x+y

defined function sum1
defined function sum2
x: Int = 55
y: Int = 210
res5_4: Int = 265

### Anonymous Function
* Same expression
    * `(x:T)=>e`
    * recursive anonymous function
        * `{ def noname(x:T) = e; (noname _) }`

In [13]:
val f1 = (x: Int) => x+1

f1: Int => Int = ammonite.$sess.cmd12$Helper$$Lambda$2112/1869084814@13503326

In [19]:
val f2 = {
    def noname(x:Int) = x+1
    noname _
}

f2: Int => Int = ammonite.$sess.cmd18$Helper$$Lambda$2151/22055630@414cc89e

In [17]:
f1(10)
f2(10)

res16_0: Int = 11
res16_1: Int = 11

In [24]:
({
    def noname(x:Int) = x+1
    noname _
}).apply(10)
((x: Int) => x+1).apply(10)

res23_0: Int = 11
res23_1: Int = 11

In [25]:
                        // E1[]
val t = 0               // E1[t=0]
def f(x: =>Int) = t + x // E1[t=0, f=(x)t+x]
val r = {               // E1[t=0, f=(x)t+x]:E2[]
    val t = 10          // E1[t=0, f=(x)t+x]:E2[t=10]
    f(t*t)              // E1[t=0, f=(x)t+x]:E2[t=10]
                 // f(t*t) E1[t=0, f=(x)t+x]:E3[x=(t*t,E2)], t+x ~ 0+x
                      // x E1[t=0, f=(x)t+x]:E2[t=10], t*t ~ 100
}                       // E1[t=0, f=(x)t+x, r=100]

t: Int = 0
defined function f
r: Int = 100

### Currying & Uncurrying
##### `(Int, Int) => Int`
##### `Int => (Int => Int)`

In [26]:
def sum(f: Int=>Int, a: Int, b: Int): Int =
if (a <= b) f(a) + sum(f, a+1, b) else 0

def sumLinear(a: Int, b: Int) = sum(n=>n, a, b)
def sumSquare(a: Int, b: Int) = sum(n=>n*n, a, b)
def sumCubes(a: Int, b: Int) = sum(n=>n*n*n, a, b)

defined function sum
defined function sumLinear
defined function sumSquare
defined function sumCubes

### *Why not?* 
> Use currying!

In [27]:
def sum(f: Int=>Int): (Int,Int)=>Int = {
    def sumF(a: Int, b: Int): Int =
        if (a <= b) f(a) + sumF(a+1, b) else 0
    sumF
}

defined function sum

In [28]:
def sumLinear = sum(n=>n)
def sumSquare = sum(n=>n*n)
def sumCubes = sum(n=>n*n*n)

defined function sumLinear
defined function sumSquare
defined function sumCubes

##### `(T1, T2, ..., Tn) => T`
##### `T1 => (T2 => ... (Tn => T))`

In [29]:
def sum(f: Int=>Int)(a: Int, b: Int): Int =
    if (a <= b) f(a) + sum(f)(a+1, b) else 0

defined function sum

In [32]:
def sumLinear = sum(n=>n) _
def sumSquare = sum(n=>n*n) _
def sumCubes = sum(n=>n*n*n) _

defined function sumLinear
defined function sumSquare
defined function sumCubes